### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)

In [4]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product

phase = '_reshape'
conds = ['no_odor','odorall']
###############################################################################
st = study('respi')
path_npz = join(st.path, 'figure/0_Classif_Power_Odor_NoOdor_Reshaped_1000perm_40pts_filter/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}{}_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
freqs = ['0_VLFC','1_delta','2_theta', '3_alpha','4_beta','5_gamma2']
min_win = 1
th = 0.01

#Find the permutation threshold by frequency band
perm_max = np.array([])
for freq in freqs:
    mat = np.load(f_form.format('All_subjects',freq,phase))
    s_perm = mat['s_perm']
    print(s_perm.shape)
    for elec in range(s_perm.shape[0]):
        perm = s_perm[elec][:,np.newaxis]
        th_perm = perm_pvalue2level(perm, p=th, maxst=True)
        perm_max = np.hstack((perm_max,th_perm[0])) if np.size(perm_max) else th_perm[0]
    print(freq, max(perm_max))
print('perm max size',perm_max.shape)
thresh = max(perm_max) #find the max of perm thr across time and subjects

# Create masks for a specific perm threshold
for freq in freqs:
    mat = np.load(f_form.format('All_subjects',freq,phase))
    s_da = mat['s_da']
    print('da',s_da.shape,np.max(s_da))
    mask = np.array([])
    for elec in range(s_da.shape[0]):
        da_elec = s_da[elec]
        pvals = np.array([])
        for win in range(da_elec.shape[0]):
            if da_elec[win] > thresh:
                pval = 1
            if da_elec[win] <= thresh:
                pval = 0
            pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
        #criteria for an elec to be significant
        pvals = np.ravel(pvals)
        underp = np.where(pvals == 1)[0]
        pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
        signif = [True for k in pvsplit if len(k) >= min_win]
        if len(signif) >=1:
            remove = False
        else:
            remove = True
        mask = np.hstack((mask, remove)) if np.size(mask) else remove
    print(freq, len(mask),len(np.where(mask == False)[0]))
    np.save(f_save.format('All_subjects',freq,min_win,th,'freqs'),mask)

-> respi loaded
(765, 1000, 40)
0_VLFC 0.592628205128
(765, 1000, 40)
1_delta 0.595192307692
(765, 1000, 40)
2_theta 0.595192307692
(765, 1000, 40)
3_alpha 0.598397435897
(765, 1000, 40)
4_beta 0.598397435897
(765, 1000, 40)
5_gamma2 0.598397435897
perm max size (183600,)
da (765, 40) 0.653653846154
0_VLFC 765 16
da (765, 40) 0.645929487179
1_delta 765 22
da (765, 40) 0.675256410256
2_theta 765 44
da (765, 40) 0.653397435897
3_alpha 765 42
da (765, 40) 0.633076923077
4_beta 765 21
da (765, 40) 0.66744047619
5_gamma2 765 15


### 2- Create masks - min patients

In [5]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product
from PIL import Image

conds, phase = ['no_odor','odorall'],'_reshape'
th, corr = '0.01', False
###############################################################################
st = study('respi')
path_classif = join(st.path, 'classified/0_Classif_Power_Odor_NoOdor_Reshaped_1000perm_40pts_filter/')
path_npz = join(st.path, 'figure/0_Classif_Power_Odor_NoOdor_Reshaped_1000perm_40pts_filter/')
if corr == True:
    path_rois, path_stats = join(path_npz,'Signif_ROIs'+th+'/'), join(path_npz, 'masks_stat_corr/')
else :
    path_rois, path_stats = join(path_npz,'Signif_ROIs'+th+'/'), join(path_npz, 'masks_stat/')
path_to_save = join(path_npz, 'masks_visbrain'+th+'/')
path_save_fig = join(path_npz, 'Signif_timecourse'+th+'_corr/') if corr == True else join(path_npz, 'Signif_timecourse'+th+'/')
npz_form = join(path_npz, '{}_sources_{}{}_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz')
rois_form = join(path_rois, '{}_signif_min_{}_{}_min_win{}_th_{}_corr.npy') if corr == True else join(path_rois, '{}_signif_min_{}_{}_min_win{}_th_{}.npy')
stat_form = join(path_stats,'{}_mask_stat_{}_minwin{}_th{}.npy')
mask_vis_form = join(path_to_save,'{}_mask_min{}_{}_minwin{}_th{}_corr.npy') if corr == True else join(path_to_save,'{}_mask_min{}_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
if not exists(path_save_fig):
    makedirs(path_save_fig)
############################################################################### 

freqs = ['0_VLFC','1_delta','2_theta', '3_alpha', '4_beta','5_gamma2']
su_codes = {'S0':'CHAF','S1':'FERJ','S2':'LEFC','S3':'MICP','S4':'PIRJ','S5':'SEMC','S6':'VACJ'} #'S6':'MICP'
min_wins = ['1'] #,'2','3','4','5']
methods = ['MAI_RL'] #,'aal_RL','BA','aal',
min_sigs = [1] #1,2,3,4,

for freq,min_win,min_sig,method in product(freqs,min_wins,min_sigs,methods):
    # Load the electrodes labels, electrodes significant (bool), rois significant (names)
    npz_mat = np.load(npz_form.format('All_subjects',freq,phase))
    print(npz_mat.files) #, npz_mat['su_codes'], npz_mat['su_names'])
    labels = npz_mat['s_labels']
    subjects, names = npz_mat['su_codes'], npz_mat['su_codes']
    elecs, channels = npz_mat['s_elec'],npz_mat['s_channels']
    stats = np.load(stat_form.format('All_subjects',freq,min_win,th))
    filename_rois = rois_form.format('s_Mai_RL',min_sig,freq[2:],min_win,th)
    if exists(filename_rois):
        rois = np.load(filename_rois)
        mask_vis = []
        i=0
        for e in range(labels.shape[0]):
            if stats[e]==0 and labels[e] in rois:
                mask_vis.append(False)
                i=i+1
                print(subjects[e], names[e],labels[e],channels[e],elecs[e],'number of elecs',i)
                #path_file = path_classif+su_codes[subjects[e]]+'/'+freq+'/fig/'
                path_file_time = path_classif+freq+'/fig/'
                filename = su_codes[subjects[e]]+'_Power'+phase+'_'+conds[0]+'_'+conds[1]+'_'+labels[e]+'_('+elecs[e]+').png'
                img = Image.open(path_file_time+filename)
                path_save_fig2 = path_save_fig+freq+'/'+labels[e]+'/'
                if not exists(path_save_fig2):
                    makedirs(path_save_fig2)
                img.save(path_save_fig2+'/min_patient'+str(min_sig)+'_minwin_'+str(min_win)+'_'+filename,'png')
            else :
                mask_vis.append(True)
            #print(stats[e],labels[e], mask_vis[e])
        np.save(mask_vis_form.format(method,min_sig,freq,min_win,th),mask_vis)

-> respi loaded
['s_elec', 'su_codes', 's_aal', 's_elec_pow0', 's_BA', 's_aal_RL', 's_elec_pow1', 's_labels', 's_da', 's_perm', 's_channels', 's_xyz']
S2 S2 RSC g5-g4 33 number of elecs 1
S3 S3 MTG a'7-a'6 4 number of elecs 2
S3 S3 MTG a'8-a'7 5 number of elecs 3
S3 S3 MTG a'9-a'8 6 number of elecs 4
S3 S3 MTG b'8-b'7 20 number of elecs 5
S3 S3 MTG b'9-b'8 21 number of elecs 6
S3 S3 MTG b'10-b'9 22 number of elecs 7
S3 S3 HC d'6-d'5 40 number of elecs 8
S3 S3 MTG d'11-d'10 43 number of elecs 9
S3 S3 MTG d'12-d'11 44 number of elecs 10
S3 S3 Ins h4-h3 47 number of elecs 11
S3 S3 STG h9-h8 52 number of elecs 12
S3 S3 STG h'6-h'5 58 number of elecs 13
S3 S3 STG h'7-h'6 59 number of elecs 14
S3 S3 Ins t'4-t'3 85 number of elecs 15
S3 S3 STG y'5-y'4 94 number of elecs 16
['s_elec', 'su_codes', 's_aal', 's_elec_pow0', 's_BA', 's_aal_RL', 's_elec_pow1', 's_labels', 's_da', 's_perm', 's_channels', 's_xyz']
S3 S3 MTG a'7-a'6 4 number of elecs 1
S3 S3 MTG a'8-a'7 5 number of elecs 2
S3 S3 MTG a'

### Max threshold by subjects

In [6]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product

phase = '_reshape'
conds = ['no_odor','odorall']
###############################################################################
st = study('respi')
path_npz = join(st.path, 'figure/0_Classif_Power_Odor_NoOdor_Reshaped_1000perm_40pts_filter/')
path_to_save = join(path_npz, 'masks_stat_by_su/')
f_form = '{}_sources_{}{}_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
n_perms = 1000
min_win = 2.
freqs = ['0_VLFC','1_delta','2_theta', '3_alpha','4_beta','5_gamma2']
th = 0.01

#Define parameters
mat = np.load(f_form.format('All_subjects','2_theta',phase))
s_su = mat['su_codes']
subjects = np.unique(s_su)
print('subjects',subjects)

#Find threshold by subject
th_by_su = []
for su in subjects:
    th_su = []
    for freq in freqs:
        mat = np.load(f_form.format('All_subjects',freq,phase))
        id_su = np.where(mat['su_codes']==su)
        s_perm = mat['s_perm'][id_su]
        th_elecs = []
        for elec in range(s_perm.shape[0]):
            #print(s_perm.shape, s_perm[elec].shape, s_perm[elec].size)
            if s_perm[elec].size > n_perms:
                perm = s_perm[elec]
            else:
                perm = s_perm[elec][:,np.newaxis]
            th_perm = perm_pvalue2level(perm, p=th, maxst=True)[0]
            th_elecs.append(th_perm)
        th_su.append(max(th_elecs))
    max_su = max(th_su)
    th_by_su.append(max_su)
print('perm max size',th_by_su)

-> respi loaded
subjects ['S0' 'S1' 'S2' 'S3' 'S4' 'S5' 'S6']
perm max size [0.57205882352941173, 0.53276397515527951, 0.53293128654970767, 0.59839743589743599, 0.52690886699507389, 0.52916666666666667, 0.53155172413793106]


In [10]:
from collections import Counter
# Create masks for a specific perm threshold
for freq in freqs:
    mat = np.load(f_form.format('All_subjects',freq,phase))    
    print(mat.files)
    mask_f, labels = np.array([]),[]
    for i,su in enumerate(subjects):
        id_su = np.where(mat['su_codes']==su)
        #print('before select ROIs',mat['s_da'].shape, np.size(mat['s_da'][1]))
        if np.size(mat['s_da'][1]):
            s_da = mat['s_da'][id_su]
        else:
            s_da = mat['s_da'][id_su][:,np.newaxis]
        #print('after select ROIs',s_da.shape)
        s_labels = mat['s_labels'][id_su]
        n_elecs, npts = s_da.shape
        mask_su = []
        for elec in range(n_elecs):
            da_elec = s_da[elec]
            label = s_labels[elec]
            pvals = np.array([])
            for t in range(npts):
                if da_elec[t] > th_by_su[i]:
                    pval = 1
                    labels.append(label)
                if da_elec[t] <= th_by_su[i]:
                    pval = 0
                pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= min_win]
            if len(signif) >=1:
                remove = False
            else:
                remove = True
            mask_su.append(remove)
        #print(labels)
        mask_f = np.hstack((mask_f, mask_su)) if np.size(mask_f) else mask_su
    print(freq, len(mask_f),len(np.where(mask_f == False)[0]),Counter(labels))
    np.save(f_save.format('All_subjects',freq,min_win,th,'freqs'),mask_f)

['s_elec', 'su_codes', 's_aal', 's_elec_pow0', 's_BA', 's_aal_RL', 's_elec_pow1', 's_labels', 's_da', 's_perm', 's_channels', 's_xyz']
0_VLFC 765 38 Counter({'MTG': 233, 'HC': 94, 'SMG': 60, 'RSC': 41, 'Ins': 35, 'ITG': 30, 'TTG': 16, 'STG': 16, 'Amg-PirT': 7, 'FuG': 7, 'PoG': 6, 'SFMG': 3, 'pPirT': 1})
['s_elec', 'su_codes', 's_aal', 's_elec_pow0', 's_BA', 's_aal_RL', 's_elec_pow1', 's_labels', 's_da', 's_perm', 's_channels', 's_xyz']
1_delta 765 59 Counter({'MTG': 241, 'HC': 128, 'PoG': 38, 'STG': 38, 'ITG': 32, 'SMG': 22, 'RSC': 11, 'PHG': 9, 'MFG': 6, 'Ins': 4, 'IFG': 4, 'OFC': 3, 'TTG': 2, 'FuG': 2})
['s_elec', 'su_codes', 's_aal', 's_elec_pow0', 's_BA', 's_aal_RL', 's_elec_pow1', 's_labels', 's_da', 's_perm', 's_channels', 's_xyz']
2_theta 765 109 Counter({'SMG': 259, 'MTG': 187, 'PoG': 170, 'STG': 150, 'HC': 77, 'TTG': 54, 'Pcun': 37, 'IFG': 30, 'ITG': 27, 'PHG': 19, 'Ins': 18, 'RSC': 16, 'OFC': 15, 'FuG': 14, 'SFMG': 10, 'PrG': 9, 'Amg': 3, 'AnG': 2, 'MCC': 2, 'PCC': 2})
['s_el